### Self-Ask with Search Agents

**Definition:**

- This approach allows the AI to ask itself follwo-up questions when it doesn't immediately know the answer. it performs a recursive questioning process to breakdown complete queries into smaller, more manageable ones.
- Once the AI generates these follow-up questions, it performs an external search(e.g. through google or an internal database) to gather the necessary information before formulating the response.

**How it works:**

- Step 1: Receive a complex questions
- Step 2: The agents identifies sub-questions or follow-up questions
- Step 3: It promotes a search or fetches answers to these questions from external resources (like a web search) 
- Step 4: The answers are aggregated to provide a complete response.

**Key Points:**

- Emphasizes question decomposition
- Relies in external search for sub-questions.
- Useful for answering open-ended or broad questions when the answer is not immediately available.

**Examples:**

- original question:"How many moons does jupiter have?"
- sub-question: "what is jupiter?"(search)
- sub-question: "what is moons?"(search)
- Finally, it retrieves the answer:"Jupiter has 79 moons"

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["SERPER_API_KEY"]=SERPER_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [10]:
## Loading the model and Embeddings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings= GoogleGenerativeAIEmbeddings(model='models/embedding-001')

llm= ChatGoogleGenerativeAI(model='gemini-1.0-pro')

In [11]:
from langchain.tools.tavily_search import TavilySearchResults

search= TavilySearchResults(max_results=1, name="Intermediate Answer")

tools= [search]

In [12]:
from langchain import hub

prompt= hub.pull("hwchase17/self-ask-with-search")

print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [14]:
from langchain.agents import create_self_ask_with_search_agent
from langchain.agents import AgentExecutor

agent= create_self_ask_with_search_agent(llm, tools, prompt)

agent_executor= AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

response= agent_executor.invoke({'input':'which country in the world have more number of languages and how many languages?'})
print(response)



> Entering new AgentExecutor chain...
Yes.
Follow up: How many languages are spoken in the world?[{'url': 'https://www.nationsonline.org/oneworld/languages.htm', 'content': 'How many languages are there in the world? There are about 7000 spoken languages in the world and about 300 writing systems. Which language is the number 1 worldwide? According to various sources, English is the most widely spoken language in the world, with around 1.35 billion speakers (first and second language (L1 and L2)).'}]Could not parse output: Follow up: Which country has the highest number of languages?
Invalid or incomplete responseFollow up: How many languages are spoken in Papua New Guinea?[{'url': 'https://en.wikipedia.org/wiki/Languages_of_Papua_New_Guinea', 'content': 'Papua New Guinea, a sovereign state in Oceania, is the most linguistically diverse country in the world. [5] According to Ethnologue, there are 840 living languages spoken in the country. [6] In 2006, Papua New Guinea Prime Minister

In [16]:
response= agent_executor.invoke({'input':'who is tamilnadu cheif minister when India got freedom ?'})
print(response)



> Entering new AgentExecutor chain...
Yes
Follow up: When did India get freedom?[{'url': 'https://www.britannica.com/topic/Indian-Independence-Movement', 'content': 'Ask the Chatbot Games & Quizzes History & Society Science & Tech Biographies Animals & Nature Geography & Travel Arts & Culture Money Videos On January 26, 1930, the Indian National Congress publicly declared its Purna Swaraj (“Complete Self-Rule”) resolution, decisively rejecting the idea of dominion status within the British Empire and establishing full sovereignty as the goal of the independence movement. Sarojini Naidu was a distinguished figure in Indian politics and literature, sometimes known as the “Nightingale of India.” She played a leading role in the Salt March and later became president (1925–26) of the Indian National Congress, becoming the first Indian woman to do so.'}]Intermediate answer: 15th August 1947
Follow up: who was tamilnadu cheif minister then?[{'url': 'https://www.thecurrentindia.com/govt/chie